# Settings

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ROOT = "/content/drive/25-2-Machine-Learning-Onions/"
import sys
sys.path.append(ROOT)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score

from src.train_tuning import Model
from src.evaluation_binary import show_confusion_matrix, show_roc_curve, show_pr_curve, show_prediction_score

is_binary = True
model_wrapper = Model(is_binary=is_binary)

# Load Data & Model

In [ ]:
with open(f"./data/preprocessed/open_test_33.pkl", "rb") as f:
    val = pickle.load(f)
    X_test = pd.DataFrame(val["X"])
    X_test.columns = val["feature_names"]
    y_test = val["y_binary"]
    y_test = np.where(y_test == -1, 0, y_test)

print("Loading complete.")
print("Test data size:\t", len(X_test))

In [ ]:
model_pkl = "./models/binary/stacked.pkl
model_wrapper.load_model(model_pkl)
final_model = model_wrapper._model

# Evaluation

In [ ]:
n_classes = len(np.unique(y_test))
class_names = [str(i) for i in range(n_classes)]

y = y_test
y_pred = final_model.predict(X_test)
y_prob = final_model.predict_proba(X_test)[:, 1]

In [ ]:
model_scores = {}

# 1. Base Model들의 Accuracy 계산
for name, model in zip(final_model.named_estimators_.keys(), final_model.estimators_):
    base_y_pred = model.predict(X_test)
    model_scores[name] = accuracy_score(y, base_y_pred)
# 2. Stacking Model의 Accuracy 계산
model_scores['Stacking Model'] = accuracy_score(y, y_pred)


plt.figure(figsize=(10, 6))

names = list(model_scores.keys())
values = list(model_scores.values())
colors = ['gray'] * (len(names) - 1) + ['navy']
bars = plt.bar(names, values, color=colors)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, height, f'{height:.4f}', ha='center', va='bottom')

plt.title('Accuracy Comparison: Base Models vs Stacking Model')
plt.ylabel('Accuracy')
plt.ylim(min(values) - 0.05, 1.05)
plt.show()

In [ ]:
show_confusion_matrix(y, y_pred, "final_model_open")

In [ ]:
show_roc_curve(y, y_prob, "final_model_open")

In [ ]:
show_pr_curve(y, y_prob, "final_model_open")

In [ ]:
show_prediction_score(y, y_prob, "final_model_open")